In [1]:
## imports
import pandas as pd
import numpy as np
import mne
import re
import os

from mne.preprocessing import (
    ICA, 
    create_ecg_epochs, 
    corrmap, 
    create_eog_epochs
)

# mne graphics config
# from PySide6 import QtCore, QtGui, QtWidgets, __version__ 

rawa = None
rawb = None

import datetime
print('IMPORTED MODULES today:', datetime.datetime.now())

# Removed cached function import
import sys
try:
    del sys.modules['rename_events']
except KeyError:
    pass

from rename_events import relabel_events


# setup

# SESSION AND POSNER CODES
session_id='m_03_01'
posner_id ='pos1a'
posner_id_b ='pos1b'

IMPORTED MODULES today: 2023-07-17 15:11:56.478528


In [2]:
%matplotlib qt

# Posner A

In [3]:
# Relabel stim events and load raw data
raw_a = relabel_events(session_id, posner_id)

Data file:  m_03_01/m_03_01_pos1a.vhdr


/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 20', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109', 'Stimulus/S117']
Found '1a' in file: m_03_01/03_1a_posner_task_2023-04-26_14h01.45.589/03_1a_posner_task_2023-04-26_14h01.45.589.csv
Found '1b' in file: m_03_01/03_1b_posner_task_2023-04-26_14h57.54.855/03_1b_posner_task_2023-04-26_14h57.54.855.csv
********** 1a
Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 20', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109', 'Stimulus/S117']
Cropping annotations 2023-04-26 14:04:04.496147+00:00 - 2023-04-26 14:16:18.396147+00:00
  [0] Keeping  (2023-04-26 14:04:04.496147+00:00 - 2023-04-26 14:04:04.496147+00:00 -> 0.0 - 0.0)
  [1] Keeping  (2023-04-26 14:04:46.884147+00:00 - 2023-04-26 1

/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


In [4]:
event_map = {'1': 11, '10': 10, '101': 101, '109': 109, '11': 11, '117': 117, '20': 20, '30': 30, '40': 40, '5': 5, '50': 50, '51': 51, '52': 52, '60': 60, '70': 70, '80': 80, '99999': 99999}

# get events and times
events, event_id = mne.events_from_annotations(raw_a, event_map)

e_a = events
# all rows with fixation (code 40)
e_40_a = e_a[e_a[:, 2] == 40]
# all rows with trial end (code 101)
e_101_a = e_a[e_a[:,2]==101]

a_tmin= e_40_a[0][0]/1000
a_tmax= e_101_a[-1][0]/1000

# Crop and make a copy and filter
filt_raw_a = raw_a.crop(tmin=a_tmin, tmax=a_tmax).copy().filter(l_freq=1.0, h_freq=40)

epochs_a = mne.Epochs(filt_raw_a, 
                    e_a, 
                    event_id=None, 
                    tmin=-0.2, 
                    tmax=0.5, 
                    preload=True, 
                    baseline=(None, 0))

Used Annotations descriptions: ['10', '101', '109', '117', '20', '30', '40', '50', '51', '52', '60', '70', '80', '99999']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Not setting metadata
415 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 415 events and 701 original time points ...
6 bad epochs dropped


[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.6s finished


In [50]:
raw_a.plot()

Using pyopengl with version 3.1.6


Channels marked as bad:
none
Channels marked as bad:
none


In [5]:
# Fitting ICA
ica = ICA(n_components=len(filt_raw_a.ch_names)-1, max_iter="auto", random_state=999)
ica.fit(filt_raw_a)
ica

Fitting ICA to data using 63 channels (please be patient, this may take a while)
Selecting by number: 63 components
Fitting ICA took 34.0s.


Method,fastica
Fit,55 iterations on raw data (667375 samples)
ICA components,63
Available PCA components,63
Channel types,eeg
ICA components marked for exclusion,—


In [9]:
ica.plot_components()

[<MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 585x260 with 3 Axes>]

In [ ]:
raw_a.load_data()
ica.plot_sources(raw_a)

Creating RawArray with float64 data, n_channels=63, n_times=667375
    Range : 52069 ... 719443 =     52.069 ...   719.443 secs
Ready.
Using qt as 2D backend.
Using pyopengl with version 3.1.6


    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
333 matching events found
No baseline correction applied
0 projection items activated


In [10]:
print(ica.exclude)

[44, 30, 31, 25, 0, 26]


In [11]:
reconstr_raw = raw_a.copy()
# Apply ica to copy
ica.apply(reconstr_raw)
# Plot
raw_a.plot(title="Raw data")
reconstr_raw.plot(title="ICA applied")

Applying ICA to Raw instance
    Transforming to ICA space (63 components)
    Zeroing out 6 ICA components
    Projecting back using 63 PCA components
Using pyopengl with version 3.1.6
Using pyopengl with version 3.1.6


Channels marked as bad:
none
Channels marked as bad:
none


In [12]:
# Save file
file_name = session_id+'_'+posner_id+'.vhdr'
output_dir='cleaned'

print(file_name)
cur_dir = os.getcwd()
session_dir = os.path.join(cur_dir, output_dir, session_id)
print(session_dir)

if not os.path.exists(session_dir):
   os.makedirs(session_dir)

print("\n**** Saving file to: ", session_dir+'/'+file_name) 
mne.export.export_raw(session_dir+'/'+file_name, reconstr_raw, verbose=1)

m_03_01_pos1a.vhdr
/Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_03_01

**** Saving file to:  /Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_03_01/m_03_01_pos1a.vhdr


/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/_export.py:61: RuntimeWarning: Encountered data in 'short' format. Converting to float32.
  warn(
/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/io.py:690: UserWarning: Encountered unsupported non-voltage units: n/a
Note that the BrainVision format specification supports only µV.
  warn(msg)


In [13]:
!tree {session_dir}

/Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_03_01
├── m_03_01_pos1a.eeg
├── m_03_01_pos1a.vhdr
└── m_03_01_pos1a.vmrk

1 directory, 3 files


In [14]:
del raw_a
del reconstr_raw
del ica
del filt_raw_a

# Posner B

In [15]:
# Relabel stim events and load raw data
raw_b = relabel_events(session_id, posner_id_b)

Data file:  m_03_01/m_03_01_pos1b.vhdr


/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109']
Found '1a' in file: m_03_01/03_1a_posner_task_2023-04-26_14h01.45.589/03_1a_posner_task_2023-04-26_14h01.45.589.csv
Found '1b' in file: m_03_01/03_1b_posner_task_2023-04-26_14h57.54.855/03_1b_posner_task_2023-04-26_14h57.54.855.csv
********** 1b
Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109']
Cropping annotations 2023-04-26 14:59:51.241112+00:00 - 2023-04-26 15:12:25.801112+00:00
  [0] Keeping  (2023-04-26 14:59:51.241112+00:00 - 2023-04-26 14:59:51.241112+00:00 -> 0.0 - 0.0)
  [1] Keeping  (2023-04-26 15:00:32.335112+00:00 - 2023-04-26 15:00:32.335112+00:00 -> 41.094 - 41.094)
  [2] Keeping  (2023-04-26 

/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


  [272] Keeping  (2023-04-26 15:08:01.119112+00:00 - 2023-04-26 15:08:01.119112+00:00 -> 489.878 - 489.878)
  [273] Keeping  (2023-04-26 15:08:03.769112+00:00 - 2023-04-26 15:08:03.769112+00:00 -> 492.528 - 492.528)
  [274] Keeping  (2023-04-26 15:08:03.869112+00:00 - 2023-04-26 15:08:03.869112+00:00 -> 492.628 - 492.628)
  [275] Keeping  (2023-04-26 15:08:06.119112+00:00 - 2023-04-26 15:08:06.119112+00:00 -> 494.878 - 494.878)
  [276] Keeping  (2023-04-26 15:08:07.619112+00:00 - 2023-04-26 15:08:07.619112+00:00 -> 496.378 - 496.378)
  [277] Keeping  (2023-04-26 15:08:11.069112+00:00 - 2023-04-26 15:08:11.069112+00:00 -> 499.828 - 499.828)
  [278] Keeping  (2023-04-26 15:08:11.169112+00:00 - 2023-04-26 15:08:11.169112+00:00 -> 499.928 - 499.928)
  [279] Keeping  (2023-04-26 15:08:12.619112+00:00 - 2023-04-26 15:08:12.619112+00:00 -> 501.378 - 501.378)
  [280] Keeping  (2023-04-26 15:08:14.119112+00:00 - 2023-04-26 15:08:14.119112+00:00 -> 502.878 - 502.878)
  [281] Keeping  (2023-04-26

In [16]:
event_map = {'1': 11, '10': 10, '101': 101, '109': 109, '11': 11, '117': 117, '20': 20, '30': 30, '40': 40, '5': 5, '50': 50, '51': 51, '52': 52, '60': 60, '70': 70, '80': 80, '99999': 99999}

# get events and times
events_b, event_id_b = mne.events_from_annotations(raw_b, event_map)

e_b = events_b
# all rows with fixation (code 40)
e_40_b = e_b[e_b[:, 2] == 40]
# all rows with trial end (code 101)
e_101_b = e_b[e_b[:,2]==101]

b_tmin= e_40_b[0][0]/1000
b_tmax= e_101_b[-1][0]/1000

# Crop and make a copy and filter
filt_raw_b = raw_b.crop(tmin=b_tmin, tmax=b_tmax).copy().filter(l_freq=1.0, h_freq=40)

epochs_b = mne.Epochs(filt_raw_b, 
                    e_b, 
                    event_id=None, 
                    tmin=-0.2, 
                    tmax=0.5, 
                    preload=True, 
                    baseline=(None, 0))

Used Annotations descriptions: ['10', '101', '109', '30', '40', '50', '51', '52', '60', '70', '80', '99999']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Not setting metadata
412 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 412 events and 701 original time points ...
4 bad epochs dropped


[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.6s finished


In [5]:
raw_b.plot()

Using qt as 2D backend.
Using pyopengl with version 3.1.6


Channels marked as bad:
none


In [17]:
# Fitting ICA
ica_b = ICA(n_components=len(filt_raw_b.ch_names)-1, max_iter="auto", random_state=999)
ica_b.fit(filt_raw_b, verbose='DEBUG')
ica_b

Fitting ICA to data using 63 channels (please be patient, this may take a while)
Selecting by number: 63 components
Cropping annotations 2023-04-26 15:00:38.936112+00:00 - 2023-04-26 15:11:52.369112+00:00
  [0] Keeping  (2023-04-26 15:00:38.936112+00:00 - 2023-04-26 15:00:38.936112+00:00 -> 47.695 - 47.695)
  [1] Keeping  (2023-04-26 15:00:40.436112+00:00 - 2023-04-26 15:00:40.436112+00:00 -> 49.195 - 49.195)
  [2] Keeping  (2023-04-26 15:00:43.206112+00:00 - 2023-04-26 15:00:43.206112+00:00 -> 51.965 - 51.965)
  [3] Keeping  (2023-04-26 15:00:43.306112+00:00 - 2023-04-26 15:00:43.306112+00:00 -> 52.065 - 52.065)
  [4] Keeping  (2023-04-26 15:00:45.436112+00:00 - 2023-04-26 15:00:45.436112+00:00 -> 54.195 - 54.195)
  [5] Keeping  (2023-04-26 15:00:46.936112+00:00 - 2023-04-26 15:00:46.936112+00:00 -> 55.695 - 55.695)
  [6] Keeping  (2023-04-26 15:00:50.486112+00:00 - 2023-04-26 15:00:50.486112+00:00 -> 59.245 - 59.245)
  [7] Keeping  (2023-04-26 15:00:50.586112+00:00 - 2023-04-26 15:00

Method,fastica
Fit,72 iterations on raw data (673433 samples)
ICA components,63
Available PCA components,63
Channel types,eeg
ICA components marked for exclusion,—


In [22]:
ica_b.plot_components()

[<MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 585x260 with 3 Axes>]

In [ ]:
raw_b.load_data()
ica_b.plot_sources(raw_b)

Creating RawArray with float64 data, n_channels=63, n_times=673433
    Range : 47695 ... 721127 =     47.695 ...   721.127 secs
Ready.
Using pyopengl with version 3.1.6


    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
336 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
336 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
336 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
336 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
336 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
336 matching events found
No baseline correction applied
0 projection items ac

In [25]:
ica_b.exclude

[54, 21, 32, 0, 13, 4, 14]

In [26]:
reconstr_raw_b = raw_b.copy()
# Apply ica to copy
ica_b.apply(reconstr_raw_b)
# Plot and compare
raw_b.plot(title="Raw data (posner b)")
reconstr_raw_b.plot(title="ICA applied (posner b)")

Applying ICA to Raw instance
    Transforming to ICA space (63 components)
    Zeroing out 7 ICA components
    Projecting back using 63 PCA components
Using pyopengl with version 3.1.6
Using pyopengl with version 3.1.6


Channels marked as bad:
none
Channels marked as bad:
none


In [27]:
# Save file
file_name_b = session_id+'_'+posner_id_b+'.vhdr'
output_dir_b='cleaned'
cur_dir = os.getcwd()
session_dir = os.path.join(cur_dir, output_dir_b, session_id)
if not os.path.exists(session_dir):
   os.makedirs(session_dir)

print("\n**** Saving file to: ", session_dir+'/'+file_name_b) 
mne.export.export_raw(session_dir+'/'+file_name_b, reconstr_raw_b, verbose=1)


**** Saving file to:  /Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_03_01/m_03_01_pos1b.vhdr


/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/_export.py:61: RuntimeWarning: Encountered data in 'short' format. Converting to float32.
  warn(
/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/io.py:690: UserWarning: Encountered unsupported non-voltage units: n/a
Note that the BrainVision format specification supports only µV.
  warn(msg)


In [28]:
!tree {session_dir}

/Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_03_01
├── m_03_01_pos1a.eeg
├── m_03_01_pos1a.vhdr
├── m_03_01_pos1a.vmrk
├── m_03_01_pos1b.eeg
├── m_03_01_pos1b.vhdr
└── m_03_01_pos1b.vmrk

1 directory, 6 files


# ...

In [29]:
del raw_b
del reconstr_raw_b
del ica_b
del filt_raw_b